In [1]:
import torch
import numpy as np
import pandas as pd

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [3]:
#from transformers import BertTokenizer, BertModel, AutoModelForSequenceClassification, AdamW, get_scheduler
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')                                                     
#model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=655)

In [4]:
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, AdamW, get_scheduler
model_name = "sberbank-ai/sbert_large_nlu_ru"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2648)
#попробуем позже

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/sbert_large_nlu_ru and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
#load best model
model.load_state_dict(torch.load("../../../best_model2306/pytorch_model.bin", map_location=device))

<All keys matched successfully>

In [7]:
torch.cuda.empty_cache()
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [12]:
df = pd.read_csv("../data/mergedcleared1307.csv", sep=';', names=['id', 'label'], dtype={'id': str, 'label': str})

In [13]:
len(df['id'].unique())

2648

In [12]:
dscr = pd.read_csv("../data/desc.csv", sep=';', names=['id', 'label'])
# ids = pd.Series(dscr['id'])
# ids = ids.astype(str)
# dscr['id'] = ids
# ids = []
# for i, id in enumerate(dscr['id']):
#     if len(id) == 4:
#         ids.append(str(id))
#     else:
#         ids.append('0' + str(id))
# dscr['id'] = ids

In [13]:
withlabels = list(dscr['id'])
justids = list(df['g_id'].unique())
for each in justids:
    if each not in withlabels:
        print(each, len(df[df['g_id']==each]))

1062 2
1018 5
1012 5
1068 1
5022 8
5058 1
5032 1
5035 1
5052 1
5075 2
6908 1


In [14]:
df = df[df['label'].notnull()]

In [15]:
from sklearn import preprocessing
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.fit(df['id'])

LabelEncoder()

In [16]:
df['labels'] = Label_encoder.fit_transform(df['id'])

In [17]:
df.columns = ['id', 'description', 'labels']


In [18]:
df = df.drop(['id'], axis=1, inplace=False)
df

,description,labels
0,удочки рыболовные разборные состоящие из отдел...,2581
1,удочки рыболовные разборные состоящие из отдел...,2581
2,удочки рыболовные штекерные карбоновые длина у...,2581
3,удочки рыболовные составные состоящие из отдел...,2581
4,удочки рыболовные телескопические выдвижные с...,2581
...,...,...
232971,электрическое осветительное оборудование,2468
232972,части мебели из прочих материалов,2442
232973,части мебели из древесины,2440
232974,части мебели для сидения,2414


In [19]:
df.to_csv('../../cler_enc1307.csv', index=False)

In [20]:
np.save('./cl_classes1307.npy', Label_encoder.classes_)

In [21]:
df = df.sample(frac=1).reset_index(drop=True)
train = df[:int(len(df)*0.8)]
test =  df[int(len(df)*0.8):]

In [22]:
print(len(test['labels'].value_counts()))

2535


In [23]:
train.to_csv('../../../train.csv', index=False)
test.to_csv('../../../test.csv', index=False)

In [5]:
#!pip install datasets

In [8]:
from datasets import load_dataset, load_metric
dataset = load_dataset('csv', data_files={'train': '../../../train.csv', 'test': '../../../test.csv'})

Found cached dataset csv (/home/sincosxy/.cache/huggingface/datasets/csv/default-0821f906fffb8bd9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset = dataset.map(lambda e: tokenizer(e['description'], truncation = True, max_length=100, padding='max_length'), batched=True)

Loading cached processed dataset at /home/sincosxy/.cache/huggingface/datasets/csv/default-0821f906fffb8bd9/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-0b384cf38472dc19.arrow


Map:   0%|          | 0/46596 [00:00<?, ? examples/s]

In [10]:
pytorch_style_columns = ['input_ids', 'token_type_ids', 'attention_mask', 'labels']  # 
dataset = dataset.remove_columns(set(dataset['train'].features.keys()) - set(pytorch_style_columns))

In [12]:
set(dataset['train'].features.keys())


{'attention_mask', 'input_ids', 'labels', 'token_type_ids'}

In [13]:
dataset.set_format(type='torch', columns=pytorch_style_columns, device='cuda') 

In [14]:
dataset.shape

{'train': (186380, 4), 'test': (46596, 4)}

In [15]:
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset['train'], shuffle=True, batch_size=12)
test_dataloader = DataLoader(dataset['test'], shuffle=False, batch_size=12)
#если есть видеопамять около 10гб, можно поставить 8

In [32]:
len(train_dataloader)

15532

In [16]:
optimizer = AdamW(model.parameters(), lr=2e-6)
num_epochs = 15
num_training_steps = num_epochs * len(train_dataloader)

lr_scheduler = get_scheduler(
    "constant_with_warmup",
    optimizer=optimizer,
    num_warmup_steps=2000,
    num_training_steps=num_training_steps
)

/home/sincosxy/.local/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [17]:
torch.cuda.empty_cache()
import gc
gc.collect()

69

In [18]:
from tqdm import tqdm

In [19]:
#next(iter(train_dataloader))
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(120138, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

In [21]:
best_f1 = 0.
show_train_loss_every_num_epoch = 0.1

for epoch in range(num_epochs):
    print(40*'-', '\nepoch', epoch+1)
    model.train()
    losses = []

    for i, batch in enumerate(tqdm(train_dataloader)):
        model.train()
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        losses.append(loss.item())

        #f i%int(len(train_dataloader)*show_train_loss_every_num_epoch)==int(len(train_dataloader)*show_train_loss_every_num_epoch)-1:
    print(f'train loss [{i*100/len(train_dataloader):.2f}%]: {np.array(losses).mean():.3f}')
    losses = []
    print('\nvalidating')

    f1 = load_metric('f1')
    acc = load_metric('accuracy')
    precision = load_metric('precision')
    recall = load_metric('recall')
    with torch.no_grad():
        model.eval()
                
        for batch in tqdm(test_dataloader):
        ##for batch in test_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            with torch.no_grad():
                outputs = model(**batch)
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            f1.add_batch(predictions=predictions, references=batch["labels"])    
            acc.add_batch(predictions=predictions, references=batch["labels"])    
            precision.add_batch(predictions=predictions, references=batch["labels"])    
            recall.add_batch(predictions=predictions, references=batch["labels"])    

        print('weighted summary:')
        print('Test acc:', acc.compute()['accuracy'])
        print('Test precision:', precision.compute(average = 'weighted')['precision'])
        print('Test recall:', recall.compute(average = 'weighted')['recall'])
        f1_weighted = f1.compute(average = 'weighted')['f1']
        print('Test f1:', f1_weighted, '\n')

        if f1_weighted > best_f1:
            best_f1 = f1_weighted
            model.save_pretrained("../../../best_model2306")

    model.train()

---------------------------------------- 
epoch 1


100%|█████████████████████████████████████████████| 15532/15532 [1:31:08<00:00,  2.84it/s]
/tmp/ipykernel_3252/608282909.py:25: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  f1 = load_metric('f1')


train loss [99.99%]: 1.234

validating


100%|█████████████████████████████████████████████████| 3883/3883 [06:30<00:00,  9.95it/s]


weighted summary:
Test acc: 0.8148982745300025


...

weighted summary:
Test acc: 0.8882522104901708


/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test precision: 0.8931073729677269
Test recall: 0.8882522104901708
Test f1: 0.8841729239392733 

---------------------------------------- 
epoch 7


  7%|███▍                                          | 1148/15532 [06:44<1:24:28,  2.84it/s]


KeyboardInterrupt: 

In [22]:
from sklearn import preprocessing
from sklearn.metrics import classification_report
Label_encoder = preprocessing.LabelEncoder()
Label_encoder.classes_ = np.load('./cl_classes1307.npy', allow_pickle=True)

true = []
preds = []

model.eval()
for batch in tqdm(test_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    true += batch["labels"].detach().cpu().numpy().tolist()
    preds += predictions.detach().cpu().numpy().tolist()

print(classification_report(Label_encoder.inverse_transform(true), Label_encoder.inverse_transform(preds)))

100%|█████████████████████████████████████████████████| 3883/3883 [06:23<00:00, 10.12it/s]
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _

              precision    recall  f1-score   support

  0207149909       1.00      1.00      1.00        14
  0304999800       1.00      1.00      1.00        14
  0305542000       0.95      1.00      0.98        20
  0305549000       1.00      1.00      1.00         5
  0305599000       0.94      0.94      0.94        16
  0604909900       1.00      1.00      1.00        12
  0701905000       1.00      1.00      1.00        21
  0703200000       1.00      0.90      0.95        10
  0711400000       1.00      1.00      1.00        11
  0711590000       1.00      1.00      1.00         7
  0712909000       1.00      1.00      1.00        15
  0713339000       1.00      1.00      1.00        15
  0801110000       1.00      1.00      1.00        17
  0802310000       0.95      1.00      0.98        20
  0802320000       1.00      0.96      0.98        27
  0803901000       1.00      1.00      1.00        18
  0804300009       0.93      1.00      0.97        14
  0804500001       1.00    

/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sincosxy/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
